Data: https://www.phila.gov/property/data/

Reference: https://medium.com/@GaussEuler/philadelphia-housing-data-part-i-data-analysis-fe45415554a9

### Mount Google Drive to access file

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/

In [ ]:
!Powershell.exe -Command "Invoke-WebRequest -Uri 'https://phl.carto.com/api/v2/sql?q=SELECT+*,+ST_Y(the_geom)+AS+lat,+ST_X(the_geom)+AS+lng+FROM+opa_properties_public&filename=opa_properties_public&format=csv&skipfields=cartodb_id' -OutFile 'data.csv'"


### Set up pandas with CSV file

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
all_data = pd.read_csv('data.csv')

In [ ]:
all_data.shape

### See all data columns

In [ ]:
sorted(all_data.columns)

### Drop Items Assumed to be useless

In [ ]:
drop_data = ['the_geom', 'the_geom_webmercator','beginning_point', 'book_and_page', 'building_code_description', 'category_code_description', 'cross_reference', 'date_exterior_condition', 'mailing_address_1', 'mailing_address_2', 'mailing_care_of', 'mailing_city_state', 'mailing_street', 'market_value_date', 'owner_1', 'owner_2', 'recording_date', 'registry_number', 'state_code', 'street_code', 'street_name', 'building_code_description_new']
data = all_data.drop(drop_data, axis = 1)

### Drop Non-Residential Data

In [ ]:
# Deleting 'Vacant Land'
data = data.loc[all_data['category_code'] < 4]
#data=all_data.drop(all_data[all_data.category_code == 6].index)
# Deleting 'Commercial'
#data=data.drop(data[data.category_code == 4].index)
# Deleting 'Industrial'
#data=data.drop(data[data.category_code == 5].index)

In [ ]:
data.category_code.value_counts(dropna=False)
#Code value: 1 = Single Family
#Code value: 6 = Vacant Land
#Code value: 2 = Multi Family
#Code value: 3 = Mixed Use
#Code value: 4 = Commercial
#Code value: 5 = Industrial

### YEs

In [ ]:
data.columns

In [ ]:
print(data.value_counts(dropna=False))

### Check Each Column

In [ ]:
data.the_geom.value_counts(dropna=False)

### Check which values have missing values >25% !!! NEED TO LOOK AT COLUMNS

In [ ]:
missing_percent = data.isnull().mean() * 100
columns_missing = missing_percent[missing_percent > 20].index.tolist()
print("Columns > 25% missing", columns_missing)

#### Drop all columns that are missing more than 25% of their datapoints

In [ ]:
data = data.drop(columns_missing, axis = 1)

### Check columns again

In [ ]:
sorted(data.columns)

In [ ]:
drop_column_list = sorted(['the_geom', 'assessment_date','beginning_point', 'book_and_page', 'category_code_description',
                    'cross_reference', 'house_number','location', 'mailing_address_1', 'mailing_address_2',
                    'mailing_care_of', 'mailing_city_state', 'mailing_street', 'market_value_date','the_geom_webmercator',
                    'other_building','owner_1', 'owner_2', 'parcel_number', 'recording_date', 'sale_date',
                    'registry_number', 'unit', 'objectid','building_code', 'census_tract', 'date_exterior_condition',
                    'year_built_estimate', 'house_extension', 'mailing_zip', 'sewer', 'site_type','state_code', 'street_designation',
                    'street_name', 'street_direction', 'geographic_ward', 'depth']) #'sale_price'

# gets all attributes in drop columnlist that are not in data
diffs = [attr for attr in drop_column_list if attr not in data]

# removes these attributes from data since we dont have them
for attr in diffs:
  drop_column_list.remove(attr)

print(f"Columns to Drop: {drop_column_list}")

#list(zip(sorted(data), sorted(drop_column_list)))
data = data.drop(drop_column_list, axis = 1)



In [ ]:
print(data.value_counts(dropna=False))

In [ ]:
missing = data.isnull().sum()
missing = missing[missing >= 1000]
missing.sort_values(inplace=True)
missing.plot.barh()
plt.title("Features with missing values (nan's) > 1000")
plt.ylabel("Features")
plt.xlabel("Number of Missing values")

In [ ]:
print("\nThe most important features relative to the target - 'market_value':")
corr = data.corr()
corr.sort_values(["market_value"],ascending = False, inplace = True)
print(corr.market_value)

In [ ]:
#data.number_of_rooms.value_counts(dropna=False).head(5)

Check Missing Data

In [ ]:
missing = all_data.isnull().sum()
missing = missing[missing >= 1000]
missing.sort_values(inplace=True)
missing.plot.barh()
plt.title("Features with missing values (nan's) > 1000")
plt.ylabel("Features")
plt.xlabel("Number of Missing values")

Drop Columns with large amounts of null values or irrelevant

In [ ]:
drop_column_list = ['the_geom', 'assessment_date','beginning_point', 'book_and_page', 'category_code_description',
                    'cross_reference', 'house_number','location', 'mailing_address_1', 'mailing_address_2',
                    'mailing_care_of', 'mailing_city_state', 'mailing_street', 'market_value_date','the_geom_webmercator',
                    'other_building','owner_1', 'owner_2', 'parcel_number', 'recording_date', 'sale_date',
                    'registry_number', 'unit', 'objectid','building_code', 'census_tract', 'date_exterior_condition',
                    'year_built_estimate', 'house_extension', 'mailing_zip', 'sewer', 'site_type','state_code', 'street_designation',
                    'street_name', 'street_direction', 'geographic_ward', 'depth'] #'sale_price'
data = all_data.drop(drop_column_list, axis = 1)

Check Data Again

In [ ]:
missing = data.isnull().sum()
missing = missing[missing >= 1000]
missing.sort_values(inplace=True)
missing.plot.barh()
plt.title("Features with missing values (nan's) > 1000")
plt.ylabel("Features")
plt.xlabel("Number of Missing values")

Removing all Values in market value and sale price that are Missing

In [ ]:
data = data.dropna(subset=['market_value'])
data = data.dropna(subset=['sale_price'])

Check to see if there are still any null values

In [ ]:
data.market_value.isnull().any()


In [ ]:
data.sale_price.isnull().any()

Deleting values that are 0 for market value and sale price, since they don't make sense to have

In [ ]:
data = data.drop(data[data.market_value == 0].index)

In [ ]:
data = data.drop(data[data.sale_price == 0].index)

Isolating the Residential homes from commercial and industrial properties

In [ ]:
# Deleting 'Vacant Land'
data=data.drop(data[data.category_code == 6].index)
# Deleting 'Commercial'
data=data.drop(data[data.category_code == 4].index)
# Deleting 'Industrial'
data=data.drop(data[data.category_code == 5].index)

Fixing Longitude and Latitude

In [ ]:
data["lng"] = data.lng.fillna(data.lng.mean())

In [ ]:
data.lng.value_counts(dropna=False).head(5)

Longitude values are negative, so getting absolute value so they are positive

In [ ]:
data.loc[:, "lng"] = data['lng'].abs()

In [ ]:
data.lat.value_counts(dropna=False).head()

Latitude is fine so no need to fix

Check Zip Codes

In [ ]:
data.zip_code.value_counts(dropna=False).head()

Check Basements

In [ ]:
data.basements.value_counts(dropna=False)
# [NOTE: REPLACING 'NaN' WITH 'L' (my definition)]
# D = Full – Unknown Finish
# F = Partial - Semi-Finished
# H = Partial - Unknown Finish
# C = Full - Unfinished
# A = Full - Finished
# J = Unknown Size - Unfinished
# 0 = No Basement [NOTE: REPLACING '0' WITH 'K']
# E = Partial - Finished
# I = Unknown Size - Finished
# G = Partial - Unfinished
# B = Full - Semi-Finished

Replacing Codes

In [ ]:
data.loc[:, "basements"] = data.loc[:, "basements"].fillna('L')
data.loc[:, "basements"] = data['basements'].replace('0', 'K')

Checking Central Air

In [ ]:
data.central_air.value_counts(dropna=False)

In [ ]:
# Replacing '0' with 'N' (i.e. NO central_air)
data.loc[:, "central_air"] = data['central_air'].replace('0', 'N')
# Replacing the "NaNs" with 'N'
data.loc[:, "central_air"] = data['central_air'].fillna('N')
# Replacing '1' with 'Y' (i.e. YES central_air)
data.loc[:, "central_air"] = data['central_air'].replace('1', 'Y')

Checking Exterior Condition

In [ ]:
data.exterior_condition.value_counts(dropna=False)
# Lower values are better quality
# 0 is Missing/NA

In [ ]:
data.loc[:, "exterior_condition"] = data['exterior_condition'].fillna(0.0)

In [ ]:
data.exterior_condition.value_counts(dropna=False)

Checking Garage Type - Removing because unclear how to organize

In [ ]:
data.garage_type.value_counts(dropna=False)

In [ ]:
data = data.drop('garage_type', axis = 1)

Checking Correlation of Features to the target

In [ ]:
print("\nThe most important features relative to the target - 'market_value':")
corr = data.corr()
corr.sort_values(["market_value"],ascending = False, inplace = True)
print(corr.market_value)